#DEEP LEARNING

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import time
import os
from tqdm import tqdm

torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

Se importa librerías necesarias y configura el entorno para entrenar un modelo de deep learning con PyTorch.

In [ ]:
print("Cargando datos...")
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

user_encoder = LabelEncoder()
item_encoder = LabelEncoder()

train_users = torch.tensor(user_encoder.fit_transform(train_data['user']), dtype=torch.long)
train_items = torch.tensor(item_encoder.fit_transform(train_data['item']), dtype=torch.long)
train_ratings = torch.tensor(train_data['rating'].values, dtype=torch.float32)

user_mapping = {label: index for index, label in enumerate(user_encoder.classes_)}
item_mapping = {label: index for index, label in enumerate(item_encoder.classes_)}

def encode_with_mapping(values, mapping):
    return [mapping.get(val, len(mapping)) for val in values]

test_ids = torch.tensor(test_data['ID'].values, dtype=torch.long)
test_users = torch.tensor(encode_with_mapping(test_data['user'], user_mapping), dtype=torch.long)
test_items = torch.tensor(encode_with_mapping(test_data['item'], item_mapping), dtype=torch.long)

train_users_data, val_users_data, train_items_data, val_items_data, train_ratings_data, val_ratings_data = train_test_split(
    train_users, train_items, train_ratings, test_size=0.05, random_state=42
)

print(f"Datos de entrenamiento: {len(train_users_data)}, Datos de validación: {len(val_users_data)}")

rating_mean = train_ratings_data.mean().item()
rating_std = train_ratings_data.std().item()

def normalize_ratings(ratings):
    return (ratings - 1) / 9

def denormalize_ratings(ratings):
    return ratings * 9 + 1

train_ratings_norm = normalize_ratings(train_ratings_data)
val_ratings_norm = normalize_ratings(val_ratings_data)

train_dataset = TensorDataset(train_users_data, train_items_data, train_ratings_norm)
val_dataset = TensorDataset(val_users_data, val_items_data, val_ratings_norm)

Este bloque de código carga y prepara datos para un sistema de recomendación. Codifica usuarios e ítems con LabelEncoder, normaliza las calificaciones a un rango de 0 a 1, divide el conjunto de datos en entrenamiento y validación, y convierte todo a tensores compatibles con PyTorch (TensorDataset).

In [ ]:
class Recommender(nn.Module):
    def __init__(self, n_users, n_items, embedding_dim, dropout_rate=0.3, leaky_relu_slope=0.2):
        super(Recommender, self).__init__()
        self.user_embedding = nn.Embedding(n_users, embedding_dim)
        self.item_embedding = nn.Embedding(n_items, embedding_dim)

        nn.init.xavier_uniform_(self.user_embedding.weight)
        nn.init.xavier_uniform_(self.item_embedding.weight)

        self.dropout = nn.Dropout(dropout_rate)
        self.batch_norm1 = nn.BatchNorm1d(embedding_dim * 2)
        self.batch_norm2 = nn.BatchNorm1d(256)
        self.batch_norm3 = nn.BatchNorm1d(128)

        self.fc1 = nn.Linear(embedding_dim * 2, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)

        self.leaky_relu = nn.LeakyReLU(negative_slope=leaky_relu_slope)

    def forward(self, user, item):
        user_emb = self.user_embedding(user)
        item_emb = self.item_embedding(item)

        x = torch.cat([user_emb, item_emb], dim=1)
        x = self.batch_norm1(x)

        x = self.fc1(x)
        x = self.leaky_relu(x)  # Replaced relu with leaky_relu
        x = self.batch_norm2(x)
        x = self.dropout(x)

        x = self.fc2(x)
        x = self.leaky_relu(x)  # Replaced relu with leaky_relu
        x = self.batch_norm3(x)
        x = self.dropout(x)

        x = self.fc3(x)
        x = self.leaky_relu(x)  # Replaced relu with leaky_relu
        x = self.dropout(x)

        x = self.fc4(x)
        x = torch.sigmoid(x)

        return x.squeeze()

Este modelo (Recommender) es una red neuronal que combina embeddings de usuarios e ítems, los procesa con múltiples capas densas, normalización y dropout, y produce una predicción entre 0 y 1 mediante una función sigmoid. Está diseñado para aprender relaciones latentes entre usuarios e ítems en un sistema de recomendación.

In [9]:
n_users = len(user_encoder.classes_) + 1
n_items = len(item_encoder.classes_) + 1
embedding_dim = 64

batch_size = 2048
learning_rate = 0.001
weight_decay = 1e-5
n_epochs = 100
patience = 5

model = Recommender(n_users, n_items, embedding_dim).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

Se configuran los hiperparámetros del modelo, se define la arquitectura de la red neuronal (Recommender), y se preparan DataLoaders para el entrenamiento y validación. Se utiliza Adam como optimizador con regularización L2, y un scheduler para ajustar dinámicamente la tasa de aprendizaje cuando el rendimiento no mejora.

In [10]:
def evaluate(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for users, items, ratings in data_loader:
            users, items, ratings = users.to(device), items.to(device), ratings.to(device)
            predictions = model(users, items)
            loss = criterion(predictions, ratings)
            total_loss += loss.item() * len(ratings)
    return total_loss / len(data_loader.dataset)

La función evaluate mide el rendimiento del modelo calculando la pérdida promedio en un conjunto de datos sin actualizar parámetros. Utiliza MSELoss para comparar predicciones con valores reales y devuelve la pérdida media por muestra.

In [11]:
# Variables para early stopping
best_val_loss = float('inf')
early_stop_counter = 0
best_model_path = 'best_recommender_model.pt'

print("Iniciando entrenamiento...")
start_time = time.time()

for epoch in range(n_epochs):
    model.train()
    train_loss = 0

    train_bar = tqdm(train_loader, desc=f"Época {epoch+1}/{n_epochs}")

    for users, items, ratings in train_bar:
        users, items, ratings = users.to(device), items.to(device), ratings.to(device)

        optimizer.zero_grad()
        predictions = model(users, items)
        loss = criterion(predictions, ratings)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        train_loss += loss.item() * len(ratings)

        train_bar.set_postfix(loss=loss.item())

    train_loss /= len(train_loader.dataset)

    val_loss = evaluate(model, val_loader, criterion, device)

    scheduler.step(val_loss)

    elapsed_time = time.time() - start_time
    print(f"Época {epoch+1}/{n_epochs} | Tiempo: {elapsed_time:.2f}s | Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0
        torch.save(model.state_dict(), best_model_path)
        print(f"Modelo guardado en {best_model_path}")
    else:
        early_stop_counter += 1
        print(f"EarlyStopping: {early_stop_counter}/{patience}")

    if early_stop_counter >= patience:
        print(f"Early stopping activado después de {epoch+1} épocas")
        break

print(f"Entrenamiento completado en {time.time() - start_time:.2f} segundos")

Iniciando entrenamiento...


Época 1/100: 100%|██████████| 182/182 [00:36<00:00,  4.93it/s, loss=0.0368]


Época 1/100 | Tiempo: 38.62s | Train Loss: 0.039579 | Val Loss: 0.034157
Modelo guardado en best_recommender_model.pt


Época 2/100: 100%|██████████| 182/182 [00:35<00:00,  5.15it/s, loss=0.0274]


Época 2/100 | Tiempo: 74.90s | Train Loss: 0.027693 | Val Loss: 0.035627
EarlyStopping: 1/5


...

Época 4/100 | Tiempo: 156.03s | Train Loss: 0.015558 | Val Loss: 0.037612
EarlyStopping: 3/5


Época 5/100: 100%|██████████| 182/182 [00:43<00:00,  4.22it/s, loss=0.0157]


Época 5/100 | Tiempo: 199.54s | Train Loss: 0.011911 | Val Loss: 0.037326
EarlyStopping: 4/5


Época 6/100: 100%|██████████| 182/182 [00:37<00:00,  4.79it/s, loss=0.0111]


Época 6/100 | Tiempo: 237.93s | Train Loss: 0.008762 | Val Loss: 0.038112
EarlyStopping: 5/5
Early stopping activado después de 6 épocas
Entrenamiento completado en 237.93 segundos


Se entrena el modelo a través de múltiples épocas usando MSELoss como criterio de pérdida. Se implementa Early Stopping para detener el entrenamiento si la pérdida de validación no mejora tras varias épocas consecutivas. Además, se guarda el mejor modelo encontrado durante el entrenamiento.

In [12]:
print("Cargando el mejor modelo para predicción...")
model.load_state_dict(torch.load(best_model_path))
model.eval()

print("Generando predicciones...")
predictions = []
batch_size = 4096

with torch.no_grad():
    for i in range(0, len(test_users), batch_size):
        batch_users = test_users[i:i+batch_size].to(device)
        batch_items = test_items[i:i+batch_size].to(device)
        batch_ids = test_ids[i:i+batch_size]

        outputs = model(batch_users, batch_items)
        predicted_ratings = (denormalize_ratings(outputs) * 10).round() / 10
        predicted_ratings = predicted_ratings.clamp(1.0, 10.0)

        predictions.extend(list(zip(batch_ids.cpu().numpy(), predicted_ratings.cpu().numpy())))

predictions_df = pd.DataFrame(predictions, columns=['ID', 'rating'])
predictions_df['rating'] = predictions_df['rating'].round(1)
predictions_df.to_csv('submission-DL.csv', index=False)

print('Predicciones guardadas en submission-DL.csv')

Cargando el mejor modelo para predicción...
Generando predicciones...
Predicciones guardadas en submission-DL.csv


Se carga el mejor modelo guardado y se utiliza para predecir las calificaciones de un conjunto de prueba en lotes. Las predicciones se convierten a un formato adecuado y se exportan a un archivo CSV (submission-DL2.csv).

#CONCLUSIÓN

Aunque no fueron concebidas específicamente para sistemas de recomendación, las redes neuronales han demostrado ser herramientas excepcionalmente eficaces para esta actividad. Su versatilidad las convierte en una opción atractiva para abordar los desafíos de la recomendación, especialmente en conjuntos de datos grandes y complejos. Sin embargo, es importante reconocer que su implementación requiere un cuidadoso ajuste de hiperparámetros y entender bien el problema y como poder plantear la resolución del problema.